In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt


In [2]:
stack_fixed = 'MD657'
stack_moving = 'atlasV3'

In [3]:
warp_setting = 1
classifier_setting = 30
trial_idx = 0

# warp_setting = 5
# classifier_setting = 30
# trial_idx = 0

In [4]:
# Read transformed volumes - one structure

# structure = '7n_L'

# warped_volumes = {}
# warped_volumes[structure] = DataManager.load_transformed_volume(stack_m=stack_moving, 
#                                                                           stack_f=stack_fixed, 
#                                     classifier_setting_m=classifier_setting,
#                                     classifier_setting_f=classifier_setting,
#                                     warp_setting=warp_setting,
#                                     trial_idx=trial_idx, structure=structure)

In [4]:
# Read transformed volumes - all structures

warped_volumes = DataManager.load_transformed_volume_all_known_structures(stack_m=stack_moving, 
                                                                          stack_f=stack_fixed, 
                                    classifier_setting_m=classifier_setting,
                                    classifier_setting_f=classifier_setting,
                                    warp_setting=warp_setting,
                                    trial_idx=trial_idx, sided=True)

rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_sp5.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/score_volumes
rm -rf /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_outerContour.bp && mkdir -p /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/score_volumes


Execution failed: [Errno 12] Cannot allocate memory
[Errno 12] Cannot allocate memory
Score volume for sp5 does not exist.
Execution failed: [Errno 12] Cannot allocate memory
[Errno 12] Cannot allocate memory
Score volume for outerContour does not exist.


In [5]:
# Set colors for different contour level
levels = [0.1, 0.25, 0.5, 0.75, .99]
level_colors = {0.1: (0,255,255), 
                0.25: (0,255,0), 
                0.5: (255,0,0), 
                0.75: (255,255,0), 
                0.99: (255,0,255)}

# levels = [.5]
# level_colors = {level: (255,0,0) for level in levels}

In [6]:
# Generate overlay visualization

# estimate mapping between z and section
downsample_factor = 32
xy_pixel_distance_downsampled = XY_PIXEL_DISTANCE_LOSSLESS * downsample_factor
voxel_z_size = SECTION_THICKNESS / xy_pixel_distance_downsampled

# For getting correct contour location
xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f = \
DataManager.load_original_volume_bbox(stack=stack_fixed, vol_type='score', structure='7N', downscale=32, classifier_setting=classifier_setting)
print xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f

# Generate atlas overlay image for every section
first_sec, last_sec = metadata_cache['section_limits'][stack_fixed]
# bar = show_progress_bar(first_sec, last_sec)

0 861 0 529 103 469


# Parallel

In [9]:
def visualize_registration_one_section(sec):
    if is_invalid(stack=stack_fixed, sec=sec):
        return
    
#     bar.value = sec

#     t = time.time()
    
    img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='thumbnail', version='cropped_tif')
    
    download_from_s3_to_ec2(img_fn)
    img = imread(img_fn)
    
    viz = img.copy()
    
    z = voxel_z_size * (sec - 1) - zmin_vol_f
    z = int(np.round(z))
    
    # Find moving volume annotation contours
    for name_s, vol in warped_volumes.iteritems():
        for level in levels:
            cnts = find_contours(vol[..., z], level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 1)

    
    viz_fp = DataManager.get_alignment_viz_filepath(stack_m=stack_moving,
                                            stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=warp_setting,
                                          section=sec,
                                                   trial_idx=trial_idx)    
    create_parent_dir_if_not_exists(viz_fp)
    imsave(viz_fp, viz)
    
    upload_from_ec2_to_s3(viz_fp)
    
#     sys.stderr.write('Time: %.2f seconds\n' % (time.time() - t)) # 5s

In [11]:
t = time.time()

pool = Pool(NUM_CORES)
pool.map(visualize_registration_one_section, range(first_sec, last_sec+1))
pool.terminate()
pool.join()

sys.stderr.write('Visualize registration: %.2f seconds\n' % (time.time() - t)) # 110s

/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0082.jpg s3://mousebrainatlas-data/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0082.jpg
aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0077.jpg s3://mousebrainatlas-data/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0077.jpg
aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_do

Child returned 0
1.18 seconds.
Child returned 0
1.19 seconds.
Child returned 0
1.22 seconds.
Child returned 0
1.23 seconds.
Child returned 0
1.24 seconds.
Child returned 0
1.23 seconds.
Child returned 0
1.21 seconds.
Child returned 0
1.20 seconds.
Child returned 0
1.27 seconds.
Child returned 0
1.26 seconds.
Child returned 0
1.25 seconds.
Child returned 0
1.27 seconds.
Child returned 0
1.30 seconds.
/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
Child returned 0
1.29 seconds.
Child returned 0
1.31 seconds.
Child returned 0
1.44 seconds.


aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0093.jpg s3://mousebrainatlas-data/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0093.jpg
aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0083.jpg s3://mousebrainatlas-data/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0083.jpg
aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_do

Child returned 0
1.23 seconds.
Child returned 0
1.23 seconds.
Child returned 0
1.24 seconds.
Child returned 0
1.25 seconds.
Child returned 0
1.23 seconds.
Child returned 0
1.16 seconds.
Child returned 0
1.20 seconds.
Child returned 0
Child returned 0
1.24 seconds.
1.31 seconds.
Child returned 0
1.16 seconds.
Child returned 0
1.33 seconds.
Child returned 0
1.26 seconds.
Child returned 0
1.24 seconds.
Child returned 0
1.31 seconds.
/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
Child returned 0
1.33 seconds.
Child returned 0
1.23 seconds.
/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


...

Child returned 0
0.81 seconds.


aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0336.jpg s3://mousebrainatlas-data/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0336.jpg


Child returned 0
0.75 seconds.
Child returned 0
0.77 seconds.


aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0341.jpg s3://mousebrainatlas-data/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0341.jpg
aws s3 cp /shared/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0346.jpg s3://mousebrainatlas-data/CSHL_registration_visualization/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_0_0346.jpg


Child returned 0
0.78 seconds.
Child returned 0
0.72 seconds.
Visualize registration: 118.28 seconds


# Sequential

In [27]:
# for sec in range(first_sec, last_sec+1):
for sec in [155]:
    
    if is_invalid(stack=stack_fixed, sec=sec):
        continue
    
#     bar.value = sec

    t = time.time()
    
    img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='thumbnail', version='cropped_tif')
    download_from_s3_to_ec2(img_fn)
    img = imread(img_fn)
    
    viz = img.copy()
    
    z = voxel_z_size * (sec - 1) - zmin_vol_f
    z = int(np.round(z))
    
    # Find moving volume annotation contours
    for name_s, vol in warped_volumes.iteritems():
        for level in levels:
            cnts = find_contours(vol[..., z], level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 1)

    
#     viz_fp = DataManager.get_alignment_viz_filepath(stack_m=stack_moving,
#                                             stack_f=stack_fixed,
#                                             classifier_setting_m=classifier_setting,
#                                             classifier_setting_f=classifier_setting,
#                                             warp_setting=warp_setting,
#                                           section=sec)
    
#     create_if_not_exists(os.path.dirname(viz_fp))
#     imsave(viz_fp, viz)
    
    sys.stderr.write('Time: %.2f seconds\n' % (time.time() - t)) # 5s

Time: 0.75 seconds


In [18]:
display_image(vol[..., z])

/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':
/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float16 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/io/_io.py:132: UserWarning: tmp.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


/oasis/projects/nsf/csd395/yuncong/Brain/registration/tmp.jpg